In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
    constituency_file = "datafiles/constituency_ids.csv";
    df_constituency = pd.read_csv(constituency_file)

In [ ]:
df_constituency.head()

In [ ]:
# Create a filtered dataframe from specific columns
constituency_cols = ["CONST ID","ONS ID", "Constituency"]
df_transf_constituency= df_constituency[constituency_cols].copy()

# Rename the column headers
df_transf_constituency = df_transf_constituency.rename(columns={"CONST ID": "const_id",
                                                        "ONS ID": "ons_code",
                                                          "Constituency": "constituency_name"})

# Clean the data by dropping duplicates and setting the index
df_transf_constituency.drop_duplicates("const_id", inplace=True)
df_transf_constituency.set_index("const_id", inplace=True)

df_transf_constituency.head()

In [ ]:
connection_string = "postgres:root123@localhost:5432/ukelection_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
df_transf_constituency.to_sql(name='constituencies', con=engine, if_exists='append', index=True)

In [ ]:
connection_string = "postgres:root123@localhost:5432/ukelection_db"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()
conn.execute("delete from constituencies")
conn.execute("delete from parties")

conn.close()

# IMPORT UK ELECTION RESULTS

In [14]:
ukelection_file = "datafiles/ukelection_results.csv";
df_ukelection = pd.read_csv(ukelection_file)


In [19]:
pcs =  pd.DataFrame(df_ukelection.party_code.unique())

In [21]:
pcs.to_csv("datafiles/party_codes.csv")

In [ ]:
connection_string = "postgres:root123@localhost:5432/ukelection_db"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

df_constituencies = pd.read_sql_query("select * from constituencies", con=conn, index_col = "ons_code")

In [ ]:
df_constituencies.head()

In [ ]:
df_parties = pd.read_sql_query("select * from parties", con=conn, index_col = "party_code")

In [ ]:
df_parties.head()

In [ ]:
df_ukelection = df_ukelection.join(df_constituencies, on="ons_code").join(df_parties, on="party_code")

In [ ]:
df_ukelection.head()

In [ ]:
df_ukelection.index = [x for x in range(1, len(df_ukelection.values)+1)]

In [ ]:
df_ukelection.index.name = "id"

In [ ]:
df_ukelection.head()

In [ ]:
# Create a filtered dataframe from specific columns
df_ukelection_cols = ["const_id", "party_id", "votes_share", "year"]
df_transf_ukelection= df_ukelection[df_ukelection_cols].copy()



# Clean the data by dropping duplicates and setting the index
#df_transf_ukelection.drop_duplicates("id", inplace=True)
#df_transf_ukelection.set_index("id", inplace=True)

df_transf_ukelection.head(10)

In [ ]:
df_transf_ukelection.to_sql(name='election_results', con=engine, if_exists='append', index=True)

In [ ]:
conn.close()

# IMPORT BREXIT RESULTS


In [3]:
brexit_file = "datafiles/eu_2016_results.csv"
df_brexit = pd.read_csv(brexit_file)


In [4]:
df_brexit.head()

,ONS ID,Percent Pro-Brexit,Percent Anti-Brexit
0,E14000582,0.756499,0.243501
1,E14001011,0.712017,0.287983
2,E14000642,0.730440,0.269560
3,E14000933,0.729956,0.270044
4,E14000771,0.728316,0.271684


In [5]:
connection_string = "postgres:root123@localhost:5432/ukelection_db"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

df_constituencies = pd.read_sql_query("select * from constituencies", con=conn, index_col = "ons_code")

In [6]:
df_constituencies.head()

,const_id,constituency_name
ons_code,,
E14000582,1,Boston and Skegness
E14001011,2,Walsall North
E14000642,3,Clacton
E14000933,4,South Basildon and East Thurrock
E14000771,5,Kingston upon Hull East


In [7]:
df_brexit = df_brexit.join(df_constituencies, on="ONS ID")
df_brexit["year"] = 2016

In [8]:
df_brexit.head(100)

,ONS ID,Percent Pro-Brexit,Percent Anti-Brexit,const_id,constituency_name,year
0,E14000582,0.756499,0.243501,1,Boston and Skegness,2016
1,E14001011,0.712017,0.287983,2,Walsall North,2016
2,E14000642,0.730440,0.269560,3,Clacton,2016
3,E14000933,0.729956,0.270044,4,South Basildon and East Thurrock,2016
4,E14000771,0.728316,0.271684,5,Kingston upon Hull East,2016
5,E14000622,0.726994,0.273006,6,Castle Point,2016
6,E14000973,0.721236,0.278764,7,Stoke-on-Trent North,2016
7,E14000669,0.716507,0.283493,8,Doncaster North,2016
8,E14000717,0.715049,0.284951,9,Great Yarmouth,2016
9,E14000716,0.691597,0.308403,10,Great Grimsby,2016


In [9]:
df_brexit.index = [x for x in range(1, len(df_brexit.values)+1)]
df_brexit.index.name = "id"
df_brexit.head()

,ONS ID,Percent Pro-Brexit,Percent Anti-Brexit,const_id,constituency_name,year
id,,,,,,
1,E14000582,0.756499,0.243501,1,Boston and Skegness,2016
2,E14001011,0.712017,0.287983,2,Walsall North,2016
3,E14000642,0.730440,0.269560,3,Clacton,2016
4,E14000933,0.729956,0.270044,4,South Basildon and East Thurrock,2016
5,E14000771,0.728316,0.271684,5,Kingston upon Hull East,2016


In [10]:
# Create a filtered dataframe from specific columns
df_brexit_cols = ["const_id", "Percent Pro-Brexit", "Percent Anti-Brexit", "year"]
df_transf_brexit= df_brexit[df_brexit_cols].copy()

# Rename the column headers
df_transf_brexit = df_transf_brexit.rename(columns={"Percent Pro-Brexit": "probrexit_share",
                                                        "Percent Anti-Brexit": "antibrexit_share"})

# Clean the data by dropping duplicates and setting the index
#df_transf_ukelection.drop_duplicates("id", inplace=True)
#df_transf_ukelection.set_index("id", inplace=True)

df_transf_brexit.head(10)

,const_id,probrexit_share,antibrexit_share,year
id,,,,
1,1,0.756499,0.243501,2016
2,2,0.712017,0.287983,2016
3,3,0.730440,0.269560,2016
4,4,0.729956,0.270044,2016
5,5,0.728316,0.271684,2016
6,6,0.726994,0.273006,2016
7,7,0.721236,0.278764,2016
8,8,0.716507,0.283493,2016
9,9,0.715049,0.284951,2016


In [11]:
df_transf_brexit.to_sql(name='brexit_results', con=engine, if_exists='append', index=True)

In [12]:
conn.close()